<a href="https://colab.research.google.com/github/TKph/colab/blob/main/SNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snntorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import snntorch as snn
from snntorch import spikegen
from snntorch import utils

from torchvision import datasets, transforms

**Data**

In [ ]:
#transformの定義
transform = transforms.Compose([
            transforms.Resize((28,28)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

#データの読み込みと前処理
data_path='/tmp/data/mnist'
mnist_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
mnist_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 11515254.51it/s]


Extracting /tmp/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /tmp/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 345912.22it/s]


Extracting /tmp/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /tmp/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3202955.23it/s]


Extracting /tmp/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /tmp/data/mnist/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3144689.46it/s]

Extracting /tmp/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /tmp/data/mnist/MNIST/raw



In [ ]:
#データローダーの作成
batch_size = 128
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True)

**Model**

In [ ]:
input_dim = 28*28
hid_dim = 1000
out_dim = 10

class MySNN(nn.Module):
  def __init__(self):
    super().__init__()

    self.fc1 = nn.Linear(input_dim, hid_dim)
    self.lif1 = snn.Leaky(beta=0.95)
    self.fc2 = nn.Linear(hid_dim, out_dim)
    self.lif2 = snn.Leaky(beta=0.95)

  def forward(self, x): #x.shape = (batch x features), featuresは[0,1]正規化済み
    #initialize

    mem1 = self.lif1.init_leaky()
    mem2 = self.lif2.init_leaky()

    spk2_rec = []
    mem2_rec = []

    for step in range(steps):
      cur1 = self.fc1(x) # cur1.shape = (128, 1000)
      spk1, mem1 = self.lif1(cur1, mem1)
      cur2 = self.fc2(spk1) # cur2.shape = (128, 10)
      spk2, mem2 = self.lif2(cur2, mem2) # spk2, mem2.shape = (batch x class) spk2=[0,0,0,0,1,0,0,0,0,1], mem2=[0.2,-0.1,...,0.8]

      spk2_rec.append(spk2) # spk2, mem2.shape = (step x batch x class)
      mem2_rec.append(mem2)

    return torch.stack(spk2_rec, dim=0), torch.stack(mem2_rec, dim=0)

mysnn = MySNN()

**Metrics, Loss and Optimizer**

In [ ]:
def batch_accuracy(data, targets, train=False):
    output, _ = mysnn(data.view(batch_size, -1))
    _, idx = output.sum(dim=0).max(1)
    acc = np.mean((targets == idx).detach().cpu().numpy())

    if train:
        print(f"Train set accuracy for a single minibatch: {acc*100:.2f}%")
    else:
        print(f"Test set accuracy for a single minibatch: {acc*100:.2f}%")

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mysnn.parameters(), lr=5e-4, betas=(0.9, 0.999))

**One iteration training**

In [ ]:
steps = 25

data, targets = next(iter(train_loader))
spk_rec, mem_rec = mysnn(data.view(batch_size, -1))

loss_val = torch.zeros((1), dtype=torch.float)

for step in range(steps):
  loss_val += loss(mem_rec[step], targets)

optimizer.zero_grad()
loss_val.backward()
optimizer.step()

spk_rec, mem_rec = mysnn(data.view(batch_size, -1))
loss_val = torch.zeros((1), dtype=torch.float)

for step in range(steps):
  loss_val += loss(mem_rec[step], targets)

batch_accuracy(data, targets, train=True)

torch.Size([128, 1000]) torch.Size([128, 10])
torch.Size([128, 1000]) torch.Size([128, 10])
torch.Size([128, 1000]) torch.Size([128, 10])
Train set accuracy for a single minibatch: 8.59%


**Train**

In [ ]:
epochs = 1
steps = 25
counter = 0
train_loss_hist = []
test_loss_hist = []

for epoch in range(epochs):
  iter_counter = 0
  train_batch = iter(train_loader)

  for data, targets in train_batch:
    mysnn.train()
    #forward
    if data.view(batch_size, -1).shape[1] != 784:
      print('dataのshapeがおかしい')
      break
    spk_rec, mem_rec = mysnn(data.view(batch_size, 784))
    #loss
    loss_train = torch.zeros((1), dtype=torch.float)
    for step in range(steps):
      loss_train += loss(mem_rec[step], targets)
    #update
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()
    #loss history
    train_loss_hist.append(loss_train.item())

    #test
    with torch.no_grad():
      mysnn.eval()
      test_data, test_targets = next(iter(test_loader))
      #forward
      test_spk, test_mem = mysnn(test_data.view(batch_size, -1))
      #loss
      loss_test = torch.zeros((1), dtype=torch.float)
      for step in range(steps):
        loss_test += loss(test_mem[step], test_targets)
      test_loss_hist.append(loss_test.item())

      if counter % 50 == 0:
        batch_accuracy(data, targets, train=True)
        batch_accuracy(test_data, test_targets, train=False)
      counter += 1
      iter_counter += 1

Train set accuracy for a single minibatch: 50.00%
Test set accuracy for a single minibatch: 32.03%
Train set accuracy for a single minibatch: 89.06%
Test set accuracy for a single minibatch: 82.03%
Train set accuracy for a single minibatch: 91.41%
Test set accuracy for a single minibatch: 85.94%
Train set accuracy for a single minibatch: 92.19%
Test set accuracy for a single minibatch: 87.50%
Train set accuracy for a single minibatch: 92.19%
Test set accuracy for a single minibatch: 87.50%
Train set accuracy for a single minibatch: 92.19%
Test set accuracy for a single minibatch: 90.62%
Train set accuracy for a single minibatch: 92.97%
Test set accuracy for a single minibatch: 92.19%
Train set accuracy for a single minibatch: 90.62%
Test set accuracy for a single minibatch: 93.75%
Train set accuracy for a single minibatch: 94.53%
Test set accuracy for a single minibatch: 90.62%
Train set accuracy for a single minibatch: 93.75%
Test set accuracy for a single minibatch: 99.22%
dataのshape